## 用於評估比賽時本團隊訓練好用於最終private dataset上的模型

In [1]:
import torch
from my_data import prepare_input, prepare_label, CustomDataset, GamesDataset
from torch.utils.data import DataLoader
from my_model import ResidualBlock, ResViT, ResViT_kyu, ResViT_PS
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np

c:\Users\Josh\Anaconda3\envs\team4762\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:0" if USE_CUDA else "cpu")
print(device)

cuda:0


## 評估dan之驗證集平均準確率

In [5]:
df = open('./29_Training Dataset/Training Dataset/dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df] #把前面和下棋無關的字串分隔開，並且只取後面有關的字串
pre_data_num = 0
bs = 256
train_games, val_games = train_test_split(games, test_size=0.001, random_state=64)
val_x_list = []
val_y_list = []
val_x_pre_eight_list = []
for val_game_num, val_game in enumerate(val_games):
    val_move_list = val_game.split(',')

    for idx, val_move in enumerate(val_move_list):
        val_x = prepare_input(val_move_list[:idx])
        val_x_list.append(val_x)
        val_y = prepare_label(val_move_list[idx])
        val_y = torch.nn.functional.one_hot(val_y, 19*19)
        val_y = torch.tensor(val_y, dtype=torch.float32)
        val_y_list.append(val_y)

        if((idx+1) <= pre_data_num):
            for i in range(idx):
                val_x = torch.cat((val_x, val_x_list[idx-1-i][:2]), dim=0)
            for i in range(pre_data_num - idx):
                val_x = torch.cat((val_x, torch.zeros((2, 19, 19))), dim=0)
               
        elif((idx+1) > pre_data_num):
            for i in range(pre_data_num):
                val_x = torch.cat((val_x, val_x_list[idx-1-i][:2]), dim=0)
            
        val_x_pre_eight_list.append(val_x)


val_x_pre_eight_stack = torch.stack(val_x_pre_eight_list)
val_y_stack = torch.stack(val_y_list)

val_x_pre_eight_stack = val_x_pre_eight_stack.to(device)
val_y_stack = val_y_stack.to(device)

val_dataset = CustomDataset(val_x_pre_eight_stack, val_y_stack)
data_loader_val = DataLoader(dataset=val_dataset, batch_size=bs, shuffle=False)

model = torch.load('./team4762 final models/model2000.pth')
model.eval()
accuracy_list_top1 = []
accuracy_list_top5 = []
with torch.no_grad():  # Disable gradient calculation during validation
    for val_x, val_y in tqdm(data_loader_val):
        predictions = model(val_x)
        top_k_values, top_k_indices = torch.topk(predictions, k=5, dim=1)
        val_y_values, val_y_indices = torch.topk(val_y, k=1)
        correct_predictions_top1 = torch.eq(top_k_indices[:, 0], val_y_indices.squeeze())
        correct_predictions_top5 = torch.any(torch.eq(top_k_indices, val_y_indices), dim=1)
    
        accuracy_top1 = torch.sum(correct_predictions_top1  == True) / len(val_y)
        accuracy_top5 = torch.sum(correct_predictions_top5  == True) / len(val_y)
        accuracy_list_top1.append(accuracy_top1.item())
        accuracy_list_top5.append(accuracy_top5.item())
        
        # print(f'Accuracy_top1: {accuracy_top1 * 100:.2f}%')
        # print(f'Accuracy_top5: {accuracy_top5 * 100:.2f}%')
print(f'top 1 avg acc:{sum(accuracy_list_top1)/len(accuracy_list_top1)}')
print(f'top 5 avg acc:{sum(accuracy_list_top5)/len(accuracy_list_top5)}')

c:\AIcup_team4762\my_data.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(torch.from_numpy(x), dtype=torch.float32)
c:\Users\Josh\Anaconda3\envs\team4762\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
100%|██████████| 92/92 [00:30<00:00,  3.01it/s]

top 1 avg acc:0.5321841032608695
top 5 avg acc:0.8488002464823101


## 評估kyu之驗證集平均準確率

In [6]:
df = open('./29_Training Dataset/Training Dataset/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df] #把前面和下棋無關的字串分隔開，並且只取後面有關的字串
pre_data_num = 0
bs = 512
train_games, val_games = train_test_split(games, test_size=0.001)
val_x_list = []
val_y_list = []
val_x_pre_eight_list = []
for val_game_num, val_game in enumerate(val_games):
    val_move_list = val_game.split(',')

    for idx, val_move in enumerate(val_move_list):
        val_x = prepare_input(val_move_list[:idx])
        val_x_list.append(val_x)
        val_y = prepare_label(val_move_list[idx])
        val_y = torch.nn.functional.one_hot(val_y, 19*19)
        val_y = torch.tensor(val_y, dtype=torch.float32)
        val_y_list.append(val_y)

        if((idx+1) <= pre_data_num):
            for i in range(idx):
                val_x = torch.cat((val_x, val_x_list[idx-1-i][5]), dim=0)
            for i in range(pre_data_num - idx):
                val_x = torch.cat((val_x, torch.zeros((1, 19, 19))), dim=0)
               
        elif((idx+1) > pre_data_num):
            for i in range(pre_data_num):
                val_x = torch.cat((val_x, val_x_list[idx-1-i][5]), dim=0)
            
        val_x_pre_eight_list.append(val_x)


val_x_pre_eight_stack = torch.stack(val_x_pre_eight_list)
val_y_stack = torch.stack(val_y_list)

val_x_pre_eight_stack = val_x_pre_eight_stack.to(device)
val_y_stack = val_y_stack.to(device)

val_dataset = CustomDataset(val_x_pre_eight_stack, val_y_stack)
data_loader_val = DataLoader(dataset=val_dataset, batch_size=bs, shuffle=False)
model = torch.load('./team4762 final models/model2300.pth')
model.eval()
accuracy_list_top1 = []
accuracy_list_top5 = []
with torch.no_grad():  # Disable gradient calculation during validation
    for val_x, val_y in tqdm(data_loader_val):
        predictions = model(val_x)
        top_k_values, top_k_indices = torch.topk(predictions, k=5, dim=1)
        val_y_values, val_y_indices = torch.topk(val_y, k=1)
        correct_predictions_top1 = torch.eq(top_k_indices[:, 0], val_y_indices.squeeze())
        correct_predictions_top5 = torch.any(torch.eq(top_k_indices, val_y_indices), dim=1)

        accuracy_top1 = torch.sum(correct_predictions_top1  == True) / len(val_y)
        accuracy_top5 = torch.sum(correct_predictions_top5  == True) / len(val_y)
        accuracy_list_top1.append(accuracy_top1.item())
        accuracy_list_top5.append(accuracy_top5.item())
        
        # print(f'Accuracy_top1: {accuracy_top1 * 100:.2f}%')
        # print(f'Accuracy_top5: {accuracy_top5 * 100:.2f}%')
print(f'top 1 avg acc:{sum(accuracy_list_top1)/len(accuracy_list_top1)}')
print(f'top 5 avg acc:{sum(accuracy_list_top5)/len(accuracy_list_top5)}')


c:\Users\Josh\Anaconda3\envs\team4762\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
100%|██████████| 56/56 [00:17<00:00,  3.21it/s]

top 1 avg acc:0.5245232390505927
top 5 avg acc:0.8334691205195018


## 評估playstyle之驗證集準確率

In [7]:
df = open('./29_Training Dataset/Training Dataset/play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]
y_list = torch.tensor(game_styles)-1
y_list = torch.nn.functional.one_hot(y_list, 3)
y_list = torch.tensor(y_list, dtype = torch.float32)
bs = 64
pre_data_num = 7
train_games, val_games, train_y, val_y = train_test_split(games, y_list, test_size=0.004,random_state=64)
val_x_list = []
val_y_list = []
x = []


for val_game_num, val_game in enumerate(val_games):
    val_move_list = val_game.split(',')
    val_x = prepare_input(val_move_list)
    last_idx = len(val_move_list)-1
    if(last_idx < pre_data_num):
        for i in range(last_idx):
            val_x = torch.cat((val_x, prepare_input(val_move_list[:last_idx-1-i])[:2]), dim=0)
        for i in range(pre_data_num - last_idx):
            val_x = torch.cat((val_x, torch.zeros((2, 19, 19))), dim=0)      
    else:
        for i in range(pre_data_num):
            val_x = torch.cat((val_x, prepare_input(val_move_list[:last_idx-1-i])[:2]), dim=0)
    
    val_x_list.append(val_x)

val_x_list = torch.stack(val_x_list)

# val_y_stack = torch.stack(val_y_list)
val_x_list = val_x_list.to(device)
val_y = val_y.to(device)

val_dataset = CustomDataset(val_x_list, val_y)
data_loader_val = DataLoader(dataset=val_dataset, batch_size=len(val_games), shuffle=False)
model = torch.load('./team4762 final models/best_PS_model.pth')
model.eval()
            
total_val_loss = 0

loss_fn = torch.nn.CrossEntropyLoss()

with torch.no_grad(): 
    for val_x, val_y in tqdm(data_loader_val):
        val_outputs = model(val_x)
        val_loss = loss_fn(val_outputs, val_y)
        total_val_loss += val_loss.item() * val_x.shape[0]
        predicted_labels = torch.argmax(val_outputs, dim=1) 
        true_labels = torch.argmax(val_y, dim=1)

avg_loss_val = total_val_loss/len(val_dataset)

accuracy = torch.sum(predicted_labels == true_labels).item() / len(true_labels)


print(f'Validation Acc: {accuracy}')
print(f'Validation Loss: {avg_loss_val}')

c:\Users\Josh\Anaconda3\envs\team4762\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 1/1 [00:00<00:00,  3.70it/s]

Validation Acc: 0.7757009345794392
Validation Loss: 0.594667911529541


## 建立 public submission csv檔

In [3]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
print(coordinates)
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    # resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_numbers = [np.flip(np.argsort(prediction)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18}


In [11]:
model = torch.load('./team4762 final models/model2300.pth')
model.eval()

df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/kyu_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

test_x_list = []
for game in games:
    move_list = game.split(',')
    test_x = prepare_input(move_list)
    test_x_list.append(test_x)

test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_chars = top_5_preds_with_chars(predictions)
    
    with open('./tem4762_public_submission.csv','a') as f:
        for index in range(len(prediction_chars)):
            
            answer_row = games_id[idx*32+index] + ',' + ','.join(prediction_chars[index]) + '\n'
            f.write(answer_row)

model = torch.load('./team4762 final models/model2000.pth')
model.eval()

df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

test_x_list = []
for game in games:
    
    move_list = game.split(',')
    test_x = prepare_input(move_list)
    test_x_list.append(test_x)


test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_chars = top_5_preds_with_chars(predictions)
    
    with open('./tem4762_public_submission.csv','a') as f:
        for index in range(len(prediction_chars)):
            answer_row = games_id[idx*32+index] + ',' + ','.join(prediction_chars[index]) + '\n'
            f.write(answer_row)

# Load your own model. Here we use the baseline model
model = torch.load('./team4762 final models/best_PS_model.pth')
model.eval()

df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/play_style_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',1)[-1] for i in df]

pre_data_num=7
test_x_list = []
for game in games:
    move_list = game.split(',')
    test_x = prepare_input(move_list)
    last_idx = len(move_list)-1
    if(last_idx < pre_data_num):
        for i in range(last_idx):
            test_x = torch.cat((test_x, prepare_input(move_list[:last_idx-1-i])[:2]), dim=0)
        for i in range(pre_data_num - last_idx):
            test_x = torch.cat((test_x, torch.zeros((2, 19, 19))), dim=0)      
    else:
        for i in range(pre_data_num):
            test_x = torch.cat((test_x, prepare_input(move_list[:last_idx-1-i])[:2]), dim=0)
   
    test_x_list.append(test_x)


test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_numbers = np.argmax(predictions, axis=1)

    with open('./tem4762_public_submission.csv','a') as f:
        for index, number in enumerate(prediction_numbers):
            answer_row = games_id[idx*32+index] + ',' + str(number+1) + '\n'
            f.write(answer_row)



## 建立 private submission csv檔

In [8]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
print(coordinates)
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    # resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_numbers = [np.flip(np.argsort(prediction)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18}


In [12]:
model = torch.load('./team4762 final models/model2300.pth')
model.eval()

df = open('./29_Private Testing Dataset_Public and Private Submission Template_v2/29_Private Testing Dataset_v2/kyu_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

test_x_list = []
for game in games:
    move_list = game.split(',')
    test_x = prepare_input(move_list)
    test_x_list.append(test_x)

test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_chars = top_5_preds_with_chars(predictions)
    
    with open('./tem4762_private_submission.csv','a') as f:
        for index in range(len(prediction_chars)):
            
            answer_row = games_id[idx*32+index] + ',' + ','.join(prediction_chars[index]) + '\n'
            f.write(answer_row)

model = torch.load('./team4762 final models/model2000.pth')
model.eval()

df = open('./29_Private Testing Dataset_Public and Private Submission Template_v2/29_Private Testing Dataset_v2/dan_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

test_x_list = []
for game in games:
    move_list = game.split(',')
    test_x = prepare_input(move_list)
    test_x_list.append(test_x)


test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_chars = top_5_preds_with_chars(predictions)
    
    with open('./tem4762_private_submission.csv','a') as f:
        for index in range(len(prediction_chars)):
            answer_row = games_id[idx*32+index] + ',' + ','.join(prediction_chars[index]) + '\n'
            f.write(answer_row)

# Load your own model. Here we use the baseline model
model = torch.load('./team4762 final models/best_PS_model.pth')
model.eval()

df = open('./29_Private Testing Dataset_Public and Private Submission Template_v2/29_Private Testing Dataset_v2/play_style_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',1)[-1] for i in df]

pre_data_num=7
test_x_list = []
for game in games:
    move_list = game.split(',')
    move_list = [item for item in move_list if item != ''] #去除資料後面的空字串
    test_x = prepare_input(move_list)
    last_idx = len(move_list)-1
  
    if(last_idx < pre_data_num):
        for i in range(last_idx):
            test_x = torch.cat((test_x, prepare_input(move_list[:last_idx-1-i])[:2]), dim=0)
        for i in range(pre_data_num - last_idx):
            test_x = torch.cat((test_x, torch.zeros((2, 19, 19))), dim=0)      
    else:
        for i in range(pre_data_num):
            test_x = torch.cat((test_x, prepare_input(move_list[:last_idx-1-i])[:2]), dim=0)
   
    test_x_list.append(test_x)


test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_numbers = np.argmax(predictions, axis=1)

    with open('./tem4762_private_submission.csv','a') as f:
        for index, number in enumerate(prediction_numbers):
            answer_row = games_id[idx*32+index] + ',' + str(number+1) + '\n'
            f.write(answer_row)

